In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os
import re
import os.path
from os import path

In [ ]:
from definitions import *

In [ ]:
FWHM_factor = 2 * np.sqrt(2 * np.log(2))
def alpha_fun(delta_B, theta_0):
    return c * delta_B / (np.pi * np.tan(theta_0))

def I_envelope(y, delta_B, theta_0, wl_sigma):
    alpha = alpha_fun(delta_B, theta_0)
    E_y = np.exp(-0.5 * (2 * np.pi * alpha * wl_sigma * y) ** 2)
    return E_y
def I_empty_analytical(y, lambda_0, delta_B, theta_0, wl_sigma, up = True):
    E_y = I_envelope(y, delta_B, theta_0, wl_sigma)
    alpha = alpha_fun(delta_B, theta_0)
    # mod = np.cos(2 * np.pi * alpha * lambda_0 * y)
    p_0 = compute_p_0(delta_B, lambda_0, theta_0)
    # print(p_0)
    mod = np.cos(2 * np.pi * lambda_0 * alpha * y)
    # print(alpha * lambda_0 * y)
    # print(mod)
    if up:
        return mod * E_y
    else:
        return -mod * E_y
y = np.linspace(-0.005, 0.005, 1000)

for (folder, theta_0_deg) in [('foil_data', 5.5), ('iwsp_data', 45), ('iso_data', 20)]:
    i = 1
    y, I_up, I_down, I_empty_up, I_empty_down, By = get_data(i, folder=folder)
    print(f"Measurement {i}; By = {By}")
    y*=1e1  
    lambda_0 = 2.165e-10
    # By = 0.00625
    theta_0 = np.deg2rad(theta_0_deg)
    # I_0 = 0.45e-9
    I = I_empty_up
    indices = indices_within_range(y * 1e-3, -0.0049, 0.0049)
    def I_fit(y, I_0):
        return I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=0.1e-10) * I_0 + I_0
    popt, pcov = curve_fit(I_fit, y[indices], I[indices])
    print(popt, pcov)
    I_0 = popt[0]
    
    E_y = I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_sigma=0.1e-10) * I_0 + I_0
    p_0 = compute_p_0(By, lambda_0, theta_0)
    # plt.axvline(p_0 * 1e3, linestyle='--', color='r')
    plt.scatter(y, I_empty_up,s=5,  label='Up, empty', color='green')
    plt.plot(y, E_y, linestyle='--', label='Up, empty analytical')
    # plt.scatter(y, I_empty_down, s=2, label='Down, empty')
    plt.xlabel('y [mm]')
    plt.ylabel('I')
    plt.title(r'$I_{up,empty}$, ' + folder)
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
plt.scatter(y, I_up,s=2,  label='Up')
plt.scatter(y, I_down,s=2, label='Down')
plt.xlabel('y [mm]')
plt.ylabel('I')
plt.title('PSD intensity')
plt.grid()
plt.legend()
plt.show()

In [ ]:
import numpy.fft as fft
def plot_spectra(Is):
    indices = indices_within_range(y, -4.85, 4.85)
    N = len(indices)
    d = (y[1] - y[0])*1e-3
    f = fft.fftshift(fft.fftfreq(N, d=d))
    for I in Is:
        I_spectrum = fft.fftshift(np.abs(fft.fft(I[indices])))/N
        plt.plot(f,I_spectrum)
    f_max = 5000
    plt.xlim((-f_max,f_max))
    plt.grid()
    plt.show()
plot_spectra([I_up])
plot_spectra([I_empty_up])

In [ ]:

plot_spectra([I_up, I_down, I_empty_up, I_empty_down])
plot_spectra([I_up,I_empty_up - I_up])

In [ ]:
def compute_P_dark_field(I_up,I_down):
    return  (I_up - I_down) / (I_up + I_down)

P_s = compute_P_dark_field(I_up,I_down)
P_b = compute_P_dark_field(I_empty_up,I_empty_down)

plt.plot(y, P_s,'-',  label='Sample, analyzer')
plt.plot(y,P_b, label='No sample, analyzer')
plt.xlabel('y [mm]')

E_y = I_empty_analytical(y * 1e-3, lambda_0, By, theta_0, wl_FWHM=0.1e-10)
p_0 = compute_p_0(By, lambda_0, theta_0)
plt.plot(y, E_y)
plt.axvline(p_0 * 1e3, linestyle='--', color='r')

plt.ylabel('P')
plt.title('PSD intensity')
plt.grid()
plt.legend()
plt.show()